In [7]:
import json
import pandas as pd
import numpy as np
import ast
import os
import hashlib

In [13]:
os.system(f"cp {os.getcwd()}/temp.xlsx {os.getcwd()}/temp_copy.xlsx")

0

In [14]:
filename = 'temp_copy.xlsx'

In [15]:
data = pd.read_excel(filename)
data = data.drop('Unnamed: 0', axis = 1)

In [16]:
data['new_title'] = data['title'] + data['date'].astype(str) + data['sitting'].astype(str)

In [17]:
def create_unique_key(df, key_column = 'new_title', drop_key_column = True):
    old_columns = list(df.columns)
    df['ID'] = df[key_column].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())
    new_columns = ['ID'] + old_columns

    if drop_key_column:
        new_columns.remove(key_column)

    return df[new_columns]

In [18]:
data = create_unique_key(data)

In [19]:
deputiesVotes_columns = ['ID', 'deputiesVotes']
deputiesVotes = data[deputiesVotes_columns]

In [20]:
deputiesVotes.loc[:, 'deputiesVotes'] = deputiesVotes['deputiesVotes'].apply(ast.literal_eval)

/Users/mdebosz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [21]:
def transform_row_tuple_into_dict(current_row):

    column_names = ('deputieName','deputieParty','vote')

    row = []
    for depute in current_row:
        row.append({column_names[i] : depute[i] for i, _ in enumerate(column_names)})

    return row

In [22]:
deputiesVotes.deputiesVotes = deputiesVotes.deputiesVotes.apply(lambda x: transform_row_tuple_into_dict(x))

/Users/mdebosz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [23]:
data_columns = ['ID', 'date', 'sitting', 'title', 'voteNumber', 'voted', 'agreed',
       'against', 'stopped', 'notVoted']
votes = data[data_columns]

In [53]:
root = '../../s3/9'

In [61]:
for row in votes.itertuples():
    
    print(row[0])

    file_id = row[1]
    date_text = row[2]

    sub_path = date_text.replace('-','/')

    to_create = os.path.join(root, sub_path)
    deputiesVotes_path = os.path.join(to_create, 'deputiesVotes')

    try:
        os.makedirs(to_create)
        os.makedirs(deputiesVotes_path)
    except FileExistsError:
        print('Path exists')
    
    to_create = os.path.join(to_create, file_id+'.json')
    deputiesVotes_path = os.path.join(deputiesVotes_path, file_id+'.json')

    votes[votes.ID == file_id].to_json(to_create, orient = 'records', force_ascii=False)
    deputiesVotes[deputiesVotes.ID == file_id].to_json(deputiesVotes_path, orient = 'records', force_ascii=False)
    

0
Path exists
1
Path exists
2
Path exists
3
Path exists
4
Path exists
5
Path exists
6
Path exists
7
Path exists
8
Path exists
9
Path exists
10
Path exists
11
Path exists
12
Path exists
13
Path exists
14
Path exists
15
Path exists
16
Path exists
17
Path exists
18
Path exists
19
Path exists
20
Path exists
21
Path exists
22
Path exists
23
Path exists
24
Path exists
25
Path exists
26
Path exists
27
Path exists
28
Path exists
29
Path exists
30
Path exists
31
Path exists
32
Path exists
33
Path exists
34
Path exists
35
Path exists
36
Path exists
37
Path exists
38
Path exists
39
Path exists
40
Path exists
41
Path exists
42
Path exists
43
Path exists
44
Path exists
45
Path exists
46
Path exists
47
Path exists
48
Path exists
49
Path exists
50
Path exists
51
Path exists
52
Path exists
53
Path exists
54
Path exists
55
Path exists
56
Path exists
57
Path exists
58
Path exists
59
Path exists
60
Path exists
61
Path exists
62
Path exists
63
Path exists
64
Path exists
65
Path exists
66
Path exists
67
Pa

data